iMPORTS

In [5]:
import requests
from bs4 import BeautifulSoup

1. send a request for accessing first webpage with opinions about products


In [3]:
url = "https://www.ceneo.pl/169771365#tab=reviews"
response = requests.get(url)

2. If responce is ok, parse HTML page content into DOM structure 

In [7]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, 'html.parser')
    

3. Extract from DOM structure  opinions and their components